# Analysis of recent bank activity

- Transactions pulled from 6/1/2022 - 7/16/2022
- Only includes Checking Account Activity

In [ ]:
from abc import ABCMeta, abstractmethod
import csv
from dataclasses import dataclass, field
from datetime import datetime
from decimal import Decimal as decimal
from enum import Enum, IntEnum
from pathlib import Path
import re
from typing import Any

from pandas import DataFrame, Series
from rich import print

from models.filters import Filter, Condition, StringMatchCondition, ValueCondition, DateRangeCondition
from models.enums import Category, Column, ColumnIndex, Comparison, Month
from helpers import handle_ex
from constants import DEFAULT_DATA_DIRECTORY, DEFAULT_CLEANSED_DIRECTORY, DEFAULT_PARSED_DIRECTORY, DEFAULT_RAW_DIRECTORY

## Define Constants for working with transactions

In [ ]:
RAW_FILEPATH = Path(f"{DEFAULT_RAW_DIRECTORY}/stmt.csv")
CLEANSED_FILEPATH = Path(f"{DEFAULT_CLEANSED_DIRECTORY}/stmt.csv")
PARSED_FILEPATH = Path(f"{DEFAULT_PARSED_DIRECTORY}/stmt.csv")

COLUMN_NAMES = [Column.DATE, Column.DESCRIPTION, Column.AMOUNT, Column.RUNNING_BALANCE, Column.CATEGORY, Column.TAGS, Column.ID]
COLUMN_NAMES = [column.value for column in COLUMN_NAMES]

## Load and Clean up data
- Remove header rows
- Remove commas
- Convert empty values for amount and balance to 0.0
- Save the clensed data

In [ ]:
print(f"Opening {RAW_FILEPATH}...")
if not RAW_FILEPATH.exists():
    raise FileNotFoundError(RAW_FILEPATH)
with open(RAW_FILEPATH, 'r+') as f:
    raw_dataset = [row for row in csv.reader(f, dialect='excel')]
ZERO = decimal(0)

dataset = raw_dataset[8:]
print(f"Found {len(dataset)} rows in {RAW_FILEPATH}")
for i, row in enumerate(dataset): 
    month, day, year = row[ColumnIndex.DATE].split("/")
    row[ColumnIndex.DATE] = datetime(int(year), int(month), int(day), 0, 0, 0)
    row.extend([Category.UNCATEGORIZED, [], i+1])
    if row[ColumnIndex.AMOUNT] == "":
        row[ColumnIndex.AMOUNT] = ZERO
    else:
        amount = row[ColumnIndex.AMOUNT].replace(",","")
        row[ColumnIndex.AMOUNT] = decimal(f"{float(amount):2f}")
        if row[ColumnIndex.AMOUNT] > ZERO:(
            row)[ColumnIndex.CATEGORY] = Category.INCOME
        row[ColumnIndex.AMOUNT] = abs(row[ColumnIndex.AMOUNT])
            
    if row[ColumnIndex.RUNNING_BALANCE] == "":
        row[ColumnIndex.RUNNING_BALANCE] = ZERO
    else:
        balance = row[ColumnIndex.RUNNING_BALANCE].replace(",","")
        row[ColumnIndex.RUNNING_BALANCE] = decimal(f"{float(balance):2f}")
print(f"Outputting parsed and cleansed data to {CLEANSED_FILEPATH}...")
with open(CLEANSED_FILEPATH, "w+", newline="") as f:
    writer = csv.writer(f, dialect="excel")
    writer.writerows(dataset)
print(f"{CLEANSED_FILEPATH} saved")

## Load The DataFrame

In [ ]:
df = DataFrame(dataset, columns=COLUMN_NAMES)
print(f"There are {df.shape[0]} transactions in the dataset")

## Create Filters
- Create list of filters

In [ ]:
AMAZON_CONDITION = StringMatchCondition("AMZN")
VENMO_CONDITION = StringMatchCondition("AMZN")
filters: list[Filter] = [
    Filter("Fraud",             Category.FRAUD, [AMAZON_CONDITION, DateRangeCondition(datetime(2022, 6, 6))]),
    Filter("Fraud",             Category.FRAUD, [AMAZON_CONDITION, DateRangeCondition(datetime(2022, 7, 11))]),
    Filter("Fraud",             Category.FRAUD, [AMAZON_CONDITION, DateRangeCondition(datetime(2022, 6, 21))]),
    Filter("Chris",             Category.WEED, [VENMO_CONDITION, ValueCondition(120.0)]),
    Filter("Grubhub",           Category.DELIVERY, [StringMatchCondition("GRUBHUB")]),
    Filter("Snacks",            Category.GROCERIES, [StringMatchCondition("CHESHIRE GAS")]),
    Filter("Steam",             Category.ENTERTAINMENT, [StringMatchCondition("STEAMGAMES")]),
    Filter("McCue",             Category.MORTGAGE, [StringMatchCondition("MCCUE")]),
    Filter("AllState",          Category.INSURANCE, [StringMatchCondition("ALLSTATE")]),
    Filter("Gym",               Category.FITNESS, [StringMatchCondition("TENNIS")]),
    Filter("Gym-1",             Category.FITNESS, [StringMatchCondition("EDGE\sFITNESS")]),
    Filter("Savings",           Category.SAVINGS, [StringMatchCondition("transfer\sto\sSAV")]),
    Filter("Mortgage",          Category.SAVINGS, [StringMatchCondition("transfer\sto\sSAV")]),
    Filter("PSN",               Category.ENTERTAINMENT, [StringMatchCondition("PLAYSTATION")]),
    Filter("Dunkin",            Category.TAKEOUT, [StringMatchCondition("DUNKIN")]),
    Filter("Snacks",            Category.TAKEOUT, [StringMatchCondition("SAM'S\sFOOD")]),
    Filter("KeepTheChange",     Category.SAVINGS, [StringMatchCondition("KEEP\sTHE\sCHANGE")]),
    Filter("SavingsWithdraw",   Category.SAVINGS, [StringMatchCondition("transfer\sfrom\sSAV")]),
    Filter("Comcast",           Category.INTERNET, [StringMatchCondition("COMCAST")]),
    Filter("Att",               Category.PHONE, [StringMatchCondition("ATT\sDES")]),
    Filter("Peapod",            Category.GROCERIES, [StringMatchCondition("PEAPOD")]),
    Filter("Patreon",           Category.ENTERTAINMENT, [StringMatchCondition("PATREON\sMEMBER")]),
    Filter("Juli'sBills",       Category.LOANS, [StringMatchCondition("MAGRATH")]),
    Filter("Juli'sBills-2",     Category.LOANS, [StringMatchCondition("Magrath")]),
    Filter("PetSupplies",       Category.PETS, [StringMatchCondition("PETCO")]),
    Filter("HouseWork",         Category.GAS, [StringMatchCondition("SM\sMECHANICAL\sSERVICES")]),
    Filter("Aresco",            Category.GROCERIES, [StringMatchCondition("ARESCO")]),
    Filter("Sunoco",            Category.GROCERIES, [StringMatchCondition("SUNOCO")]),
    Filter("Affirm",            Category.LOANS, [StringMatchCondition("AFFIRM")]),
    Filter("LifeInsurance",     Category.INSURANCE, [StringMatchCondition("NEW\sYORK\sLIFE\sDES")]),
    Filter("Kindle",            Category.ENTERTAINMENT, [StringMatchCondition("KINDLE")]),
    Filter("YouTube",           Category.ENTERTAINMENT, [StringMatchCondition("YOUTUBE")]),
    Filter("Fuel",              Category.AUTO, [StringMatchCondition("CITGO")]),
    Filter("AMEXCreditCard",    Category.LOANS, [StringMatchCondition("AMERICA\sCREDIT\sCARD")]),
    Filter("MCCreditCard",      Category.LOANS, [StringMatchCondition("PAYPAL\sEXTRAS\sMASTERCARD")]),
    Filter("Bilaton",           Category.GROCERIES, [StringMatchCondition("STRYVEFOODS")]),
    Filter("Theater",           Category.ENTERTAINMENT, [StringMatchCondition("THOMASTON\sOPERA\sHOUSE")]),
    Filter("Electric",          Category.ELECTRIC, [StringMatchCondition("CL&P")]),
    Filter("PriceChopper",      Category.GROCERIES, [StringMatchCondition("PRICE\sCHOPPER")]),
    Filter("IRS",               Category.TAXES, [StringMatchCondition("IRS\sDES")]),
    Filter("Vivint",            Category.INSURANCE, [StringMatchCondition("VIVINT")]),
    Filter("Dropbox",           Category.INTERNET, [StringMatchCondition("DROPBOX")]),
    Filter("VPN",               Category.INTERNET, [StringMatchCondition("MOZILLACORP")]),
    Filter("Fairview",          Category.GIFTS, [StringMatchCondition("FAIRVIEW")]),
    Filter("UpgradeLoad",       Category.LOANS, [StringMatchCondition("UPGRADE")]),
    Filter("Eversource",        Category.GAS, [StringMatchCondition("EVERSOURCE")]),
    Filter("CarLoan",           Category.LOANS, [StringMatchCondition("CAPITAL\sONE\sAUTO")]),
    Filter("Chris",             Category.WEED, [StringMatchCondition("TO\sCHRIS")]),
    Filter("Aza",               Category.ENTERTAINMENT, [StringMatchCondition("TO\sAZA")]),
    Filter("Stryve-2",          Category.GROCERIES, [StringMatchCondition("STRYVE")]),
    Filter("GasStation",        Category.AUTO, [StringMatchCondition("FUEL\sPLUS")]),
    Filter("NetFlix",           Category.ENTERTAINMENT, [StringMatchCondition("Netflix")]), 
    Filter("AmazonFilter-1",    Category.ENTERTAINMENT, [AMAZON_CONDITION, ValueCondition(10.62)]), 
    Filter("AmazonFilter-2",    Category.GROCERIES, [AMAZON_CONDITION, ValueCondition(11.29)]),
    Filter("AmazonFilter-3",    Category.ENTERTAINMENT, [AMAZON_CONDITION, ValueCondition(53.15)]),
    Filter("AmazonFilter-4",    Category.GROCERIES, [AMAZON_CONDITION, ValueCondition(16.84)]),
    Filter("AmazonFilter-5",    Category.WEED, [AMAZON_CONDITION, ValueCondition(14.13)]),
    Filter("AmazonFilter-6",    Category.GROCERIES, [AMAZON_CONDITION, ValueCondition(26.12)]),
    Filter("AmazonFilter-7",    Category.GROCERIES, [AMAZON_CONDITION, ValueCondition(57.40)]),
    Filter("AmazonFilter-8",    Category.GROCERIES, [AMAZON_CONDITION, ValueCondition(19.30)]),
    Filter("AmazonFilter-9",    Category.ENTERTAINMENT, [AMAZON_CONDITION, ValueCondition(180.78)]),
    Filter("AmazonFilter-10",   Category.ENTERTAINMENT, [AMAZON_CONDITION, ValueCondition(17.01)]), 
]

- Automatically categorize Income as anything > $0.00 
- Run uncategorized transactions against filters, matches get category set by filter

In [ ]:
for filter in filters:
    uncategorized = df[df.category == Category.UNCATEGORIZED]
    if uncategorized.size == 0:
        break
    matches = filter.match(uncategorized)
    matches.category = filter.category
    
categorized = df[df.category != Category.UNCATEGORIZED]
uncategorized = df[df.category == Category.UNCATEGORIZED]
print(f"There are {categorized.size} categorized transactions")
print(f"There are {uncategorized.size} uncategorized transactions")

## Analysis of Categorized Transactions

In [ ]:
# CATEGORY_HEADER_TEXT = "Category"
# ZERO_TOTAL_TEXT = "$0.00"
# with open(PARSED_FILEPATH, "r+") as f:
#     trx_list = TransactionList.from_json(f.read())

# totals_by_category_by_month: dict[Month, dict[Category, float]] = {}
# months = set([Month(trx.date.month) for trx in trx_list.transactions])
# for month in months:
#     totals_by_category = {}    
#     for trx in trx_list.transactions:    
#         if trx.date.month == month:
#             if trx.category not in totals_by_category.keys():
#                 totals_by_category[trx.category] = trx.amount
#             else:
#                 totals_by_category[trx.category] += trx.amount
#     totals_by_category_by_month[month] = totals_by_category
    
# totals_by_category: dict[Category, dict[Month, float]] = {}
# for month, totals_for_month in totals_by_category_by_month.items():    
#     for category, total in totals_for_month.items():
#         if category not in totals_by_category.keys():
#             totals_by_category[category] = {Month(month): total}
#         else:
#             totals_by_category[category][Month(month)] = total

# header_row: list[str] = [
#     CATEGORY_HEADER_TEXT
# ]
# for month in months:
#     header_row.append(month.name)

# data: list[list[str]] = []
# for category, month_totals in totals_by_category.items():
#     row = [category.value]
#     for month in months:
#         if month in month_totals.keys():            
#             total = month_totals[month]
#             row.append(f"${total:.2f}")
#         else:
#             row.append(ZERO_TOTAL_TEXT)
#     data.append(row)

# DataFrame(data, columns=header_row)